In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
!pip install -U torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -U transformers==4.46.3 accelerate==1.11.0 bitsandbytes==0.45.0 triton==3.1.0 langchain==0.3.7 langchain-core==0.3.15 langchain-huggingface==0.1.2 langgraph==0.2.37

Looking in indexes: https://download.pytorch.org/whl/cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.5/768.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 MB 3.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1
  Using cached triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached torch-2.9.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010

In [10]:
# agents_graph_multi_model_optimized.py - SPECIALIZED MODELS VERSION

import os
from typing import TypedDict, Literal
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
import torch
import time

print("🔧 System Check...")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print()

# ========== STATE DEFINITION ========== #

class WorkflowState(TypedDict, total=False):
    user_instruction: str
    tone: str
    style: str
    iteration: int
    draft_text: str
    reviewed_text: str
    image_prompt: str
    compliance_status: Literal["pending", "approved", "needs_changes"]
    compliance_feedback: str
    revision_notes: str

# ========== OPTIMIZED LLM SETUP - MULTIPLE MODELS ========== #

def make_llm_quantized(model_id: str, use_4bit: bool = True):
    """Create optimized LLM with quantization"""
    from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
    from transformers import (
        AutoModelForCausalLM, 
        AutoTokenizer, 
        pipeline, 
        BitsAndBytesConfig
    )
    
    print(f"🚀 Loading: {model_id}")
    
    # Quantization config
    if use_4bit:
        print("⚡ Applying: 4-bit quantization")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
        )
    else:
        print("⚡ Applying: 8-bit quantization")
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            llm_int8_threshold=6.0,
        )
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        use_fast=True,
        padding_side='left',
        trust_remote_code=True
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Load model with optimizations
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16,
    )
    
    # Determine max tokens based on model size
    if "7b" in model_id.lower():
        max_tokens = 400  # Larger model = more capacity
    elif "phi-2" in model_id.lower():
        max_tokens = 350  # Medium capacity
    else:
        max_tokens = 300  # Smaller models
    
    # Optimized pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=max_tokens,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        batch_size=1,
    )
    # Ensure chat template exists to avoid chat_template error
    if not getattr(tokenizer, "chat_template", None):
        tokenizer.chat_template = "{% for m in messages %}{{ m['role'] }}: {{ m['content'] }}{% endfor %}"
    llm = HuggingFacePipeline(pipeline=pipe)
    return llm


print("🚀 Loading Specialized Models...")
print("\n📝 Writer Model (Zephyr-7B)...")
WRITER_LLM = make_llm_quantized("HuggingFaceH4/zephyr-7b-beta", use_4bit=True)

print("\n🔍 Reviewer Model (Phi-2)...")
REVIEWER_LLM = make_llm_quantized("microsoft/phi-2", use_4bit=True)

print("\n✅ Compliance Model (Zephyr-7B)...")
COMPLIANCE_LLM = make_llm_quantized("HuggingFaceH4/zephyr-7b-beta", use_4bit=True)

# Use Phi-2 for coordinator (fast and efficient)
print("\n🎯 Coordinator Model (Phi-2)...")
COORDINATOR_LLM = make_llm_quantized("microsoft/phi-2", use_4bit=True)

print("\n✅ All Models Ready!\n")


def chat(llm, system_prompt: str, user_prompt: str, max_input_tokens: int = 1500) -> str:
    """Optimized LLM call with strict token limits (works with HuggingFacePipeline directly)"""
    # Combine prompts
    combined = f"{system_prompt}\n\n{user_prompt}"

    # Access tokenizer from pipeline directly
    tokenizer = llm.pipeline.tokenizer

    # Truncate input
    tokens = tokenizer.encode(combined, max_length=max_input_tokens, truncation=True)
    combined = tokenizer.decode(tokens, skip_special_tokens=True)

    try:
        output = llm.pipeline(
            combined,
            max_new_tokens=300,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            temperature=0.7,
        )
        if isinstance(output, list) and "generated_text" in output[0]:
            return output[0]["generated_text"].strip()
        else:
            return str(output).strip()
    except Exception as e:
        print(f"⚠️  Error: {e}")
        return "[Generation failed]"



# ========== SPECIALIZED AGENTS ========== #

def coordinator_agent(state: WorkflowState) -> dict:
    """Fast coordinator using Phi-2"""
    iteration = state.get("iteration", 0)
    
    if iteration == 0:
        return {"iteration": 1, "revision_notes": ""}
    
    feedback = state.get("compliance_feedback", "")
    
    prompt = f"Based on this compliance feedback, list 3 key changes needed:\n{feedback}\n\nChanges:"
    
    revision_notes = chat(COORDINATOR_LLM, "You create concise revision lists.", prompt)
    
    return {
        "iteration": iteration + 1,
        "revision_notes": revision_notes,
    }


def text_generator_agent(state: WorkflowState) -> dict:
    """Generate text using Zephyr-7B (best writer)"""
    sys = "You are an expert social media copywriter. Create engaging, creative, and compelling social media content."
    
    revisions = state.get('revision_notes', '')
    prompt = f"""Create an Instagram post:
Product: {state.get("user_instruction", "")}
Tone: {state.get("tone", "")}
Style: {state.get("style", "")}
{f'Apply these revisions: {revisions}' if revisions else ''}

Write the caption:"""
    
    draft = chat(WRITER_LLM, sys, prompt, max_input_tokens=2000)
    return {"draft_text": draft}


def reviewer_agent(state: WorkflowState) -> dict:
    """Review and polish using Phi-2 (best reviewer)"""
    sys = "You are an expert editor. Review and improve the text for grammar, clarity, and engagement. Output only the edited text."
    prompt = f"Edit and improve this social media caption:\n\n{state.get('draft_text', '')}\n\nEdited version:"
    
    refined = chat(REVIEWER_LLM, sys, prompt, max_input_tokens=1800)
    return {"reviewed_text": refined}


def image_generator_agent(state: WorkflowState) -> dict:
    """Generate image prompt using Zephyr-7B"""
    sys = "You are an expert at creating detailed, vivid image prompts for AI image generation."
    prompt = f"""Create a detailed image generation prompt for this social media post:

{state.get('reviewed_text', '')}

Image prompt:"""
    
    img_prompt = chat(WRITER_LLM, sys, prompt, max_input_tokens=1500)
    return {"image_prompt": img_prompt}


def compliance_agent(state: WorkflowState) -> dict:
    """Compliance check using Zephyr-7B (best compliance)"""
    sys = """You are a content compliance officer. Check content for:
- Inappropriate language
- False claims
- Harmful content
- Brand safety issues

Reply with either:
- "APPROVED" if content is safe
- "NEEDS_CHANGES: [specific reason]" if issues found"""
    
    text = state.get('reviewed_text', '')
    img = state.get('image_prompt', '')
    
    prompt = f"""Review this content for compliance:

TEXT:
{text}

IMAGE PROMPT:
{img}

Decision:"""
    
    raw = chat(COMPLIANCE_LLM, sys, prompt, max_input_tokens=2000)
    
    lower = raw.lower()
    if "approved" in lower and "needs" not in lower:
        return {
            "compliance_status": "approved",
            "compliance_feedback": "Content approved"
        }
    else:
        return {
            "compliance_status": "needs_changes",
            "compliance_feedback": raw
        }


# ========== GRAPH - OPTIMIZED FLOW ========== #

def should_continue(state: WorkflowState) -> Literal["coordinator", "end"]:
    """Stop after approval or max iterations"""
    iteration = state.get("iteration", 0)
    status = state.get("compliance_status", "pending")
    
    # Limit to 3 iterations max
    if status == "approved" or iteration >= 3:
        return "end"
    return "coordinator"


graph = StateGraph(WorkflowState)

# Add nodes
graph.add_node("coordinator", coordinator_agent)
graph.add_node("text_generator", text_generator_agent)
graph.add_node("reviewer", reviewer_agent)
graph.add_node("image_generator", image_generator_agent)
graph.add_node("compliance", compliance_agent)

# Flow
graph.set_entry_point("coordinator")
graph.add_edge("coordinator", "text_generator")
graph.add_edge("text_generator", "reviewer")
graph.add_edge("reviewer", "image_generator")
graph.add_edge("image_generator", "compliance")
graph.add_conditional_edges("compliance", should_continue, 
                           {"coordinator": "coordinator", "end": END})

memory = MemorySaver()
app = graph.compile(checkpointer=memory)


# ========== RUN ========== #

if __name__ == "__main__":
    initial_state: WorkflowState = {
        "user_instruction": "Create an Instagram post promoting EcoWave reusable water bottles.",
        "tone": "fun, friendly, eco-conscious",
        "style": "short caption with 3-4 hashtags",
        "iteration": 0,
    }
    
    try:
        print("▶️  Starting workflow...\n")
        start = time.time()
        
        final_state = app.invoke(
            initial_state,
            config={"configurable": {"thread_id": "demo-1"}}
        )
        
        elapsed = time.time() - start
        
        print("\n" + "="*60)
        print(f"✅ DONE in {elapsed:.1f}s ({elapsed/60:.1f} min)")
        print("="*60)
        
        print("\n📝 FINAL TEXT:")
        print("-" * 60)
        print(final_state.get("reviewed_text", ""))
        
        print("\n🎨 IMAGE PROMPT:")
        print("-" * 60)
        print(final_state.get("image_prompt", ""))
        
        print("\n✅ STATUS:")
        print("-" * 60)
        print(f"Compliance: {final_state.get('compliance_status', '').upper()}")
        print(f"Iterations: {final_state.get('iteration', 0)}")
        print(f"Speed: {elapsed:.1f}s")
        
        print("\n🤖 MODELS USED:")
        print("-" * 60)
        print("Writer: HuggingFaceH4/zephyr-7b-beta (4-bit)")
        print("Reviewer: microsoft/phi-2 (4-bit)")
        print("Compliance: HuggingFaceH4/zephyr-7b-beta (4-bit)")
        print("Coordinator: microsoft/phi-2 (4-bit)")
        print("="*60 + "\n")
        
    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        import traceback
        traceback.print_exc()


# ========== PERFORMANCE NOTES ========== #
print("\n💡 OPTIMIZATION NOTES:")
print("1. First run downloads all 3 models (~10GB total)")
print("2. Zephyr-7B used for writing & compliance (best quality)")
print("3. Phi-2 used for reviewing & coordination (fast & accurate)")
print("4. All models use 4-bit quantization for speed")
print("5. GPU memory usage: ~8-10GB total")
print("6. Expected runtime: 2-4 minutes per workflow")

🔧 System Check...
PyTorch: 2.6.0+cu124
CUDA: True
GPU: Tesla T4

🚀 Loading Specialized Models...

📝 Writer Model (Zephyr-7B)...
🚀 Loading: HuggingFaceH4/zephyr-7b-beta
⚡ Applying: 4-bit quantization


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]


🔍 Reviewer Model (Phi-2)...
🚀 Loading: microsoft/phi-2
⚡ Applying: 4-bit quantization


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


✅ Compliance Model (Zephyr-7B)...
🚀 Loading: HuggingFaceH4/zephyr-7b-beta
⚡ Applying: 4-bit quantization


...


🎯 Coordinator Model (Phi-2)...
🚀 Loading: microsoft/phi-2
⚡ Applying: 4-bit quantization


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


✅ All Models Ready!

▶️  Starting workflow...



This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.



✅ DONE in 142.9s (2.4 min)

📝 FINAL TEXT:
------------------------------------------------------------
You are an expert editor. Review and improve the text for grammar, clarity, and engagement. Output only the edited text.

Edit and improve this social media caption:

You are an expert social media copywriter. Create engaging, creative, and compelling social media content.

Create an Instagram post:
Product: Create an Instagram post promoting EcoWave reusable water bottles.
Tone: fun, friendly, eco-conscious
Style: short caption with 3-4 hashtags
Apply these revisions: You create concise revision lists.

Based on this compliance feedback, list 3 key changes needed:
You are a content compliance officer. Check content for:
- Inappropriate language
- False claims
- Harmful content
- Brand safety issues

Reply with either:
- "APPROVED" if content is safe
- "NEEDS_CHANGES: [specific reason]" if issues found

Review this content for compliance:

TEXT:
You are an expert editor. Review and i